In [26]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from art.attacks.evasion import DeepFool
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np
import pandas as pd

from sklearn import metrics
from warnings import simplefilter

import joblib

In [27]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test.reshape(-1), y_pred_classes, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [28]:
x_train = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/x_train.csv', low_memory=False)
y_train = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/y_train.csv', low_memory=False)
x_test = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/x_test.csv', low_memory=False)
y_test = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/y_test.csv', low_memory=False)

x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()

In [29]:
model = joblib.load("/home/jovyan/UNSW/dl/new_dl/dnn.joblib")

2024-02-26 21:59:12.510335: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://655f38346f23487980cf0985851e57b9: INVALID_ARGUMENT: ram://655f38346f23487980cf0985851e57b9 is a directory.


In [30]:
x_test_trimmed = x_test[47700:48000]
y_test_trimmed = y_test[47700:48000]

In [31]:
np.unique(y_test_trimmed)

array([0, 1])

In [32]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=2,
    input_shape=(56,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [41]:
attack = DeepFool(classifier=classifier, epsilon=0.01)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool: 100%|██████████| 300/300 [00:04<00:00, 69.05it/s]


In [42]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 908us/step
Accuracy: 0.14333333333333334
AUC: 0.1018957345971564

macro
Precision: 0.16287878787878787
Recall: 0.1018957345971564
F1 Score: 0.12536443148688048

weighted
Precision: 0.2291161616161616
Recall: 0.14333333333333334
F1 Score: 0.17634596695821186

Mean FNR: 0.8981042654028436
Mean TNR: 0.1018957345971564
Mean FPR: 0.8981042654028436
Mean TPR: 0.1018957345971564


In [35]:
attack = DeepFool(classifier=classifier, epsilon=0.05)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool: 100%|██████████| 300/300 [00:04<00:00, 68.36it/s]


In [36]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 930us/step
Accuracy: 0.14333333333333334
AUC: 0.1018957345971564

macro
Precision: 0.16287878787878787
Recall: 0.1018957345971564
F1 Score: 0.12536443148688048

weighted
Precision: 0.2291161616161616
Recall: 0.14333333333333334
F1 Score: 0.17634596695821186

Mean FNR: 0.8981042654028436
Mean TNR: 0.1018957345971564
Mean FPR: 0.8981042654028436
Mean TPR: 0.1018957345971564


In [40]:
attack = DeepFool(classifier=classifier, epsilon=0.1)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool:   0%|          | 0/300 [00:00<?, ?it/s]

DeepFool: 100%|██████████| 300/300 [00:06<00:00, 45.51it/s]


In [43]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 870us/step
Accuracy: 0.14333333333333334
AUC: 0.1018957345971564

macro
Precision: 0.16287878787878787
Recall: 0.1018957345971564
F1 Score: 0.12536443148688048

weighted
Precision: 0.2291161616161616
Recall: 0.14333333333333334
F1 Score: 0.17634596695821186

Mean FNR: 0.8981042654028436
Mean TNR: 0.1018957345971564
Mean FPR: 0.8981042654028436
Mean TPR: 0.1018957345971564


In [44]:
attack = DeepFool(classifier=classifier, epsilon=0.15)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool: 100%|██████████| 300/300 [00:06<00:00, 46.78it/s]


In [45]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 715us/step
Accuracy: 0.14333333333333334
AUC: 0.1018957345971564

macro
Precision: 0.16287878787878787
Recall: 0.1018957345971564
F1 Score: 0.12536443148688048

weighted
Precision: 0.2291161616161616
Recall: 0.14333333333333334
F1 Score: 0.17634596695821186

Mean FNR: 0.8981042654028436
Mean TNR: 0.1018957345971564
Mean FPR: 0.8981042654028436
Mean TPR: 0.1018957345971564


In [46]:
attack = DeepFool(classifier=classifier, epsilon=0.2)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool: 100%|██████████| 300/300 [00:06<00:00, 46.27it/s]


In [47]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 782us/step
Accuracy: 0.14333333333333334
AUC: 0.1018957345971564

macro
Precision: 0.16287878787878787
Recall: 0.1018957345971564
F1 Score: 0.12536443148688048

weighted
Precision: 0.2291161616161616
Recall: 0.14333333333333334
F1 Score: 0.17634596695821186

Mean FNR: 0.8981042654028436
Mean TNR: 0.1018957345971564
Mean FPR: 0.8981042654028436
Mean TPR: 0.1018957345971564


In [48]:
attack = DeepFool(classifier=classifier, epsilon=0.25)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool: 100%|██████████| 300/300 [00:06<00:00, 46.46it/s]


In [49]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 748us/step
Accuracy: 0.14333333333333334
AUC: 0.1018957345971564

macro
Precision: 0.16287878787878787
Recall: 0.1018957345971564
F1 Score: 0.12536443148688048

weighted
Precision: 0.2291161616161616
Recall: 0.14333333333333334
F1 Score: 0.17634596695821186

Mean FNR: 0.8981042654028436
Mean TNR: 0.1018957345971564
Mean FPR: 0.8981042654028436
Mean TPR: 0.1018957345971564


In [50]:
attack = DeepFool(classifier=classifier, epsilon=0.3)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool: 100%|██████████| 300/300 [00:06<00:00, 46.55it/s]


In [51]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 749us/step
Accuracy: 0.14333333333333334
AUC: 0.1018957345971564

macro
Precision: 0.16287878787878787
Recall: 0.1018957345971564
F1 Score: 0.12536443148688048

weighted
Precision: 0.2291161616161616
Recall: 0.14333333333333334
F1 Score: 0.17634596695821186

Mean FNR: 0.8981042654028436
Mean TNR: 0.1018957345971564
Mean FPR: 0.8981042654028436
Mean TPR: 0.1018957345971564


In [54]:
attack = DeepFool(classifier=classifier, epsilon=0.001)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool: 100%|██████████| 300/300 [00:06<00:00, 46.63it/s]


In [55]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 729us/step
Accuracy: 0.14666666666666667
AUC: 0.10426540284360192

macro
Precision: 0.16541353383458646
Recall: 0.10426540284360189
F1 Score: 0.12790697674418605

weighted
Precision: 0.23268170426065163
Recall: 0.14666666666666667
F1 Score: 0.17992248062015506

Mean FNR: 0.8957345971563981
Mean TNR: 0.10426540284360189
Mean FPR: 0.8957345971563981
Mean TPR: 0.10426540284360189
